In [ ]:
import deeplabcut

In [ ]:
deeplabcut.__version__

## Step 1: create new project
### config the project path and add videos (can be manually added after the project is created)

In [ ]:
#changable
project_name='HipsDontLieBacksDo'
experimenter='Devon'
train_set=r'multimedia\videos\datafolder\camera.mov'
# better not change
working_dir = r'\DLC'


In [ ]:
config_path = r'config.yaml'

# if we want to add a new project instead of using the existing one
# config_path = deeplabcut.create_new_project(project_name,
#                                              experimenter,
#                                              [train_set],
#                                             working_directory=working_dir,
#                                              videotype='.mov',
#                                             copy_videos=True,
#                                             multianimal=True)

### print off config_path and record it for future use

In [ ]:
print(config_path)

# Skip this if creating a new model
### Re-train： Do these steps first

Delete archive files EXCEPT pose_cfg.yaml

Add videos under .\videos folder

Add the absolute paths of the videos in the configuration file

In [ ]:
config_path = r'config.yaml'

## Step 2 extract key frames 
### Re-train： if we don't want to extract key frames on the old videos repetitively (most of the cases), we need to comment out the path of the old videos in the configuration file temporarily.

In [ ]:
deeplabcut.extract_frames(config_path, mode='automatic', algo='kmeans', userfeedback=False, crop=False)

## Step 3 label the frames (manually)

In [ ]:
deeplabcut.label_frames(config_path)

### check the labels before proceeding.

In [ ]:
deeplabcut.check_labels(config_path)

## Step 4 create training dataset. 
### for single animal project
### Retrain: Here we need to go to the configuration file to un-comment the old videos path if we want to include them in the re-training

In [ ]:
deeplabcut.create_training_dataset(config_path)

### for multi animal project

In [ ]:

deeplabcut.cropimagesandlabels(config_path)

In [ ]:
deeplabcut.create_multianimaltraining_dataset(config_path)

## Step 5 train the network (usually with only one shuffle is enough)

### Re-train: this is the last step. Saved files are under .\dlc-models\iteration-0\Alec_second_tryDec7-trainset95shuffle1\train

In [ ]:
deeplabcut.train_network(config_path,
                        maxiters=200000,
                        shuffle=1,
                        max_snapshots_to_keep=10,
                        saveiters=2400,
                        displayiters=2400,
                        gputouse=0)

## Step 5 evaluate the network

In [ ]:
deeplabcut.evaluate_network(config_path)

In [ ]:
deeplabcut.extract_save_all_maps(config_path, shuffle=1, Indices=[0, 5])

In [ ]:
print(config_path)
##outputs _meta.pickle, _full.pickle
deeplabcut.analyze_videos(config_path, [r'your_path\camera_17391304.MOV'])

##doesn't stich tracklets together, just plots all tracks above confidence
deeplabcut.create_video_with_all_detections(config_path, [r'your_path\camera_17391304.MOV'])

##TODO: deeplabcut.find_outliers_in_raw_data(config_path, pickle_file, video_file) #where pickle_file is _full.pickle

##outputs _assemblies.pickle, _el.pickle
deeplabcut.convert_detections2tracklets(
    config_path, 
    [r'your_path\camera_17391304.MOV'],
    videotype='mov'
)

##TODO: use gui for refine_tracklets; also: find_outlier_in_raw_data using _assemblies.pickle


##creates a folder for this camera with labelled video/frames 
deeplabcut.utils.make_labeled_video.create_video_from_pickled_tracks(
    r'your_path\camera_17391304.MOV',
    r'your_path\camera_17391304DLC_dlcrnetms5_HipsDontLieBacksDoOct27shuffle1_200000_assemblies.pickle',
    destfolder=r"D:\\", output_name="test", pcutoff=0.6
)


In [ ]:
deeplabcut.convert_detections2tracklets(
    config_path, 
    r'your_path\camera_17391304.MOV',
    videotype='mov', overwrite=False
)
deeplabcut.refine_tracklets(
    config_path,
    r'your_path\camera_17391304DLC_dlcrnetms5_HipsDontLieBacksDoOct27shuffle1_200000_assemblies.pickle',
    r'your_path\camera_17391304.MOV',
)


## step 6 anaylze novel videos

### path of all novel videos


In [ ]:
import os
novel_video_path = r'\multimedia\videos'
video_list = os.listdir(novel_video_path)
novel_videos=[novel_video_path+ '\\'+video+'\\camera_17391304.MOV' for video in video_list]
print(novel_videos)

In [ ]:
deeplabcut.analyze_videos(config_path,
                          novel_videos,
                          save_as_csv=True,
                          videotype='mov',
                         shuffle=1)

In [ ]:
deeplabcut.create_video_with_all_detections(config_path, [novel_videos[0]])

In [ ]:
novel_videos[0]

### maDLC: detect tracklets

In [ ]:
for video in novel_videos:
    deeplabcut.convert_detections2tracklets(config_path, 
                                            video, 
                                            videotype='mov', 
                                            shuffle=1, 
                                            #trainingsetindex=0, 
                                            track_method='box')

In [ ]:
import pprint

pickle_files = []
for vid in novel_videos:
    pickle_files.append(f'{vid[:-4]}DLC_dlcrnetms5_HipsOrNeckBackDontLieOct12shuffle1_200000_bx.pickle')


pprint.pprint({k:v for k,v in zip(novel_videos,pickle_files)})

In [ ]:
# deeplabcut.refine_tracklets(config_path, pickle_files[0], novel_videos[0], min_swap_len=2, min_tracklet_len=2, trail_len=50)
deeplabcut.stitch_tracklets(config_path, novel_videos, track_method='box', n_tracks=1)


In [ ]:
deeplabcut.filterpredictions(config_path, novel_videos, videotype='MOV',track_method='box')

In [ ]:
deeplabcut.plot_trajectories(config_path, novel_videos, videotype='MOV', track_method='box')

In [ ]:
deeplabcut.create_labeled_video(config_path,novel_videos,save_frames=False,trailpoints=5,videotype='mov',draw_skeleton='True')

# The only step for exporting model for dlc-live

In [ ]:
deeplabcut.export_model(config_path,shuffle=1,
                        TFGPUinference=False,
                        make_tar=False)
